#### [ 미니 테스트  2024-03-14 ]<hr>

[1] 지도학습에 대한 설명과 종류에 대해서 간략하게 설명해 주세요.

지도학습은 데이터에 라

[2] 지도학습의 학습 프로세스를 간략하게 설명해 주세요.

[3] 선형회귀 대해 간략하게 설명해주세요.  
    - 개념  
    - 조건  
    - 동작원리  
    - 선형식  
    - 오차/잔차 계산 방법  

[4] 로지스틱회귀 대해 간략하게 설명해주세요.  
   - 개념  
   - 조건  
   - 동작원리  
   - 선형식  
   - 오차/잔차 계산 방법  

[5] Fashion MNIST 데이터셋에서 10개 카테고리 중 하나를 선택해서 2진 분류로 구현해 주세요.  
- 데이터셋 : scikit-learn의 datasets 사용
    - from sklearn.datasets import fetch_openml 
    - db_name = 'Fashion-MNIST'
    - fashion_data = fetch_openml(name=db_name, parser='auto')

In [55]:
import torch
from torch import tensor
from torch import functional as F
from torch import optim
from torch import nn

In [56]:
from sklearn.datasets import fetch_openml

db_name = 'Fashion-MNIST'
fashion_data = fetch_openml(name=db_name, parser='auto')

In [57]:
torch.manual_seed(0)

In [58]:
fashion_data

{'data':        pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
 0           0       0       0       0       0       0       0       0       0   
 1           0       0       0       0       0       1       0       0       0   
 2           0       0       0       0       0       0       0       0       0   
 3           0       0       0       0       0       0       0       0      33   
 4           0       0       0       0       0       0       0       0       0   
 ...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
 69995       0       0       0       0       0       0       0       0       0   
 69996       0       0       0       0       0       0       0       0       0   
 69997       0       0       0       0       0       0       0       0       0   
 69998       0       0       0       0       0       0       0       0       0   
 69999       0       0       0       0       0       0       0       0       0   
 
      

In [59]:
features = fashion_data['data']
target = fashion_data['target']

In [60]:
print(features['pixel1'].unique())

[ 0  7 16  1 14  2  4]


In [61]:
target = target.astype('int8')
target.unique()

array([9, 0, 3, 2, 7, 5, 1, 6, 4, 8], dtype=int8)

In [62]:
featuresT = torch.FloatTensor(features.values)
targetT = torch.FloatTensor(target.values).unsqueeze(dim=1)

In [63]:
targetT.shape, featuresT.shape

(torch.Size([70000, 1]), torch.Size([70000, 784]))

In [64]:
targetT.dtype, featuresT.dtype

(torch.float32, torch.float32)

In [65]:
for i in range(len(targetT)):
    if targetT[i] >= 5:
        targetT[i] = 1
    else:
        targetT[i] = 0
        
targetT

tensor([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [1.]])

In [66]:
sum(targetT)

tensor([35000.])

In [67]:
model = nn.Sequential(
    nn.Linear(784, 1),
    nn.Sigmoid()
)

In [68]:
for p in model.parameters():
    print(p)

Parameter containing:
tensor([[-2.6739e-04,  1.9159e-02, -2.9394e-02, -2.6284e-02, -1.3756e-02,
          9.5771e-03, -7.0762e-04,  2.8317e-02, -3.1694e-03,  9.4505e-03,
         -1.0793e-02, -7.0202e-03, -3.4120e-02, -2.3653e-02, -1.4722e-02,
          1.3230e-03,  1.4119e-02,  2.1429e-02, -2.4212e-02, -1.5552e-02,
          1.2972e-02,  2.9657e-02, -7.3500e-03,  2.6725e-02, -5.7565e-03,
          3.7791e-03,  3.2338e-02, -3.3131e-02, -2.2483e-02, -9.0416e-03,
         -1.3921e-02,  3.0857e-02, -2.3149e-02, -1.6440e-02, -2.4951e-02,
         -3.3449e-02, -2.0848e-02,  3.0700e-02,  1.5936e-02,  1.7310e-02,
          1.8783e-03, -1.8310e-02,  6.0423e-03, -3.3346e-02, -2.5806e-02,
         -1.8412e-02,  2.2533e-02,  2.0940e-02, -1.5839e-02, -1.2887e-03,
          2.2841e-02,  3.5505e-02,  1.4174e-02,  4.8247e-03,  2.3946e-02,
         -2.1029e-02,  6.6551e-03, -2.7689e-02, -2.4753e-02, -1.8449e-02,
          1.6160e-02,  1.4363e-02, -2.1155e-02,  1.0790e-02,  1.9606e-02,
         -4.5078

In [69]:
opt = optim.Adam(model.parameters(), lr=0.001)

In [70]:
max_epochs = 500

for epoch in range(1, max_epochs+1):
    h = model(featuresT)
    
    cost = torch.nn.functional.binary_cross_entropy(h, targetT)
    
    opt.zero_grad()
    cost.backward()
    opt.step()
    
    if epoch % 10 == 0:
        prediction = h >= torch.FloatTensor([0.5])
        accuracy = sum(prediction == targetT) / len(h)
        predict = sum(prediction)
        print(f' E [{epoch:>4} / {max_epochs}] ----')
        print(f'Cost : {cost.item()}, Accuracy : {accuracy.item()}, Predict : {predict.item()}')

 E [  10 / 500] ----
Cost : 8.4108304977417, Accuracy : 0.8690571188926697, Predict : 26922
 E [  20 / 500] ----
Cost : 8.472552299499512, Accuracy : 0.8415428400039673, Predict : 36676
 E [  30 / 500] ----
Cost : 7.190958023071289, Accuracy : 0.8630428314208984, Predict : 35301
 E [  40 / 500] ----
Cost : 6.36594820022583, Accuracy : 0.8747143149375916, Predict : 35276
 E [  50 / 500] ----
Cost : 5.866012096405029, Accuracy : 0.8837857246398926, Predict : 35681
 E [  60 / 500] ----
Cost : 5.208585739135742, Accuracy : 0.8914285898208618, Predict : 35764
 E [  70 / 500] ----
Cost : 4.331550598144531, Accuracy : 0.899057149887085, Predict : 35102
 E [  80 / 500] ----
Cost : 3.971658945083618, Accuracy : 0.8992571234703064, Predict : 35336
 E [  90 / 500] ----
Cost : 3.655691623687744, Accuracy : 0.9000428318977356, Predict : 35275
 E [ 100 / 500] ----
Cost : 3.4914448261260986, Accuracy : 0.9005857110023499, Predict : 35617
 E [ 110 / 500] ----
Cost : 3.2398056983947754, Accuracy : 0.90

In [71]:
from sklearn.preprocessing import OneHotEncoder

In [81]:
encoder = OneHotEncoder()
encoder.fit(targetT)
target_encode = encoder.transform(targetT).toarray()

In [82]:
targetT2 = torch.as_tensor(targetT, dtype=torch.float32)

In [83]:
max_epochs = 500

for epoch in range(1, max_epochs+1):
    h = model(featuresT)
    
    cost = torch.nn.functional.binary_cross_entropy(h, targetT2)
    
    opt.zero_grad()
    cost.backward()
    opt.step()
    
    if epoch % 10 == 0:
        prediction = h >= torch.FloatTensor([0.5])
        accuracy = sum(prediction == targetT2) / len(h)
        predict = sum(prediction)
        print(f' E [{epoch:>4} / {max_epochs}] ----')
        print(f'Cost : {cost.item()}, Accuracy : {accuracy.item()}, Predict : {predict.item()}')

 E [  10 / 500] ----
Cost : 1.9586098194122314, Accuracy : 0.9066428542137146, Predict : 30967
 E [  20 / 500] ----
Cost : 1.7732378244400024, Accuracy : 0.8971571326255798, Predict : 37807
 E [  30 / 500] ----
Cost : 0.9918172955513, Accuracy : 0.9136000275611877, Predict : 32544
 E [  40 / 500] ----
Cost : 0.9495283365249634, Accuracy : 0.9089285731315613, Predict : 35537
 E [  50 / 500] ----
Cost : 0.8451374769210815, Accuracy : 0.9103714227676392, Predict : 34774
 E [  60 / 500] ----
Cost : 0.7950634956359863, Accuracy : 0.9039714336395264, Predict : 35726
 E [  70 / 500] ----
Cost : 0.7114431858062744, Accuracy : 0.9070143103599548, Predict : 34507
 E [  80 / 500] ----
Cost : 0.7152766585350037, Accuracy : 0.9009571671485901, Predict : 35827
 E [  90 / 500] ----
Cost : 0.7603697776794434, Accuracy : 0.8920000195503235, Predict : 37322
 E [ 100 / 500] ----
Cost : 0.6510903835296631, Accuracy : 0.907542884349823, Predict : 33378
 E [ 110 / 500] ----
Cost : 0.6360321640968323, Accura